In [1]:

import os
os.environ['CUDA_VISIBLE_DEVICES']='2'
import json
import re
from datetime import datetime
import sys
import argparse

import torch
from torch.utils.data import DataLoader
from dotenv import load_dotenv

load_dotenv()
DATA_DIR = os.getenv('DATA_DIR')
CACHE_DIR = os.getenv('CACHE_DIR')
OUT_DIR = os.getenv('OUT_DIR')
FINECITE_PATH = os.getenv('FINECITE_PATH')
if FINECITE_PATH not in sys.path:
    sys.path.append(FINECITE_PATH)

from finecite.utils import set_seed, get_class_weights
from finecite.data_processing import load_processor
from finecite.model import CustomTrainer, ExtractionModel, ClassificationModel, load_classifier, load_tokenizer_embedding_model, MODEL_DESCRIPTION

In [ ]:
parser = argparse.ArgumentParser(description='Seq_tagger parser')
args = parser.parse_args([])
args.model_name = 'scibert' # scibert llm2vec_mistral llm2vec_llama3
args.ext_model = 'scibert' # scibert llm2vec_mistral llm2vec_llama3
args.dataset = 'acl-arc' # 'acl-arc', 'act2', 'scicite', 'multicite'
args.ext_type = 'bilstm_crf' # linear, bilstm, crf, bilstm_crf
args.cls_type = 'linear' # weighted, balanced, linear, inf, perc, back

args.batch_size = 4
args.learning_rate = 2e-05
args.crf_learning_rate = 0.005
args.dropout = 0.1

args.save_model = False
args.cached_data = False
args.debug = True
args.debug_size = 100
args.seed = 4455

In [3]:
args.task = 'cls'
args.max_epochs = 20
args.patients = 5
args.adam_epsilon = 1e-08
args.weight_decay = 0.0
args.use_prompt = 'llm2vec' in args.model_name
args.dtype = torch.float32

# model description
args.model_desc = MODEL_DESCRIPTION[args.model_name]
args.max_len = args.model_desc['max_len']

# input directory
args.input_dir = f'{DATA_DIR}/{args.dataset}/'

# output directory
args.output_dir = re.sub(r'[.:*?"<>|\s-]','_',(
    f"{OUT_DIR}/"
    f"{'_debug/' if args.debug else ''}"
    f"{args.dataset}/{args.model_name}/"
    f"{args.ext_type}__{args.batch_size}__{args.learning_rate}__{args.dropout}"
    f"{'__' + datetime.now().strftime('%m_%d_%H_%M_%S') if args.debug else ''}/"
))
args.model_output_file = re.sub(r'[.:*?"<>|\s]','_',(
    f"{OUT_DIR}/"
    f"{'_debug/' if args.debug else ''}"
    f"{args.dataset}/{args.model_name}/"
    f"safetensors__{args.batch_size}__{args.learning_rate}__{args.dropout}.pt"
))

os.makedirs(args.output_dir, exist_ok=True)

# model cache dir
if 'llm2vec' in args.model_name:
    args.base_model_dir =  f'{CACHE_DIR}/{args.model_name}/'
    os.makedirs(args.base_model_dir, exist_ok=True)

# data cache dir
args.data_cache_dir =  f'{CACHE_DIR}/{args.dataset}/'
os.makedirs(args.data_cache_dir, exist_ok=True)

# ext model dir
args.ext_model_dir = re.sub(r'[.:*?"<>|\s-]','_',(
    f"{OUT_DIR}/finecite/{args.ext_model}/{args.ext_type}"
))
set_seed(args.seed)
args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Extract Label

In [4]:
with open(f'{args.ext_model_dir}/run_setup.json', 'r') as f:
    ext_args =  argparse.Namespace(**json.load(f))
    
ext_args.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
ext_args.dtype = args.dtype

print('loading extraction model...')
tokenizer, embedding_model = load_tokenizer_embedding_model(ext_args)
ext_args.cls_token_id = tokenizer.cls_token_id
ext_args.sep_token_id = tokenizer.sep_token_id
ext_args.pad_token_id = tokenizer.pad_token_id

ext_model = ExtractionModel(ext_args, embedding_model)

ext_model.load_state_dict(torch.load(f'{args.ext_model_dir}/safetensors.pt'))

loading extraction model...


loading scibert model...


<All keys matched successfully>

In [ ]:
#load data processor
processor = load_processor(args, tokenizer)

# load data
train_data = processor.read_data('train')
test_data = processor.read_data('test')

# create dataset
train_ds, weights, num_labels = processor.create_features(train_data)
test_ds, _ , _ = processor.create_features(test_data)
args.label_weights = weights
args.num_labels = num_labels
args.num_training_steps = int(len(train_data) / args.batch_size) * args.max_epochs

train_file = f'{args.ext_model}_{args.ext_type}_train.pt'
test_file = f'{args.ext_model}_{args.ext_type}_test.pt'
# check cached examples
if args.cached_data and train_file in os.listdir(args.data_cache_dir) and test_file in os.listdir(args.data_cache_dir):
    train_ds = torch.load(os.path.join(args.data_cache_dir, train_file), weights_only=False)
    test_ds = torch.load(os.path.join(args.data_cache_dir, test_file), weights_only=False)
else:

    # dataloader
    train_dl = DataLoader(train_ds, batch_size = args.batch_size, shuffle=False)
    test_dl = DataLoader(test_ds, batch_size = args.batch_size, shuffle=False)
    
    #extract context
    train_lbls = ext_model.extract_token_labels(train_dl)
    test_lbls = ext_model.extract_token_labels(test_dl)
    
    assert len(train_ds) == len(train_lbls) and len(test_ds) == len(test_lbls) 
    
    #add context to dataset
    train_ds = [{**example, 'token_labels': torch.tensor(tok_lbl, dtype=torch.long)} for example, tok_lbl in zip(train_ds, train_lbls)]
    test_ds = [{**example, 'token_labels': torch.tensor(tok_lbl, dtype=torch.long)}for example, tok_lbl in zip(test_ds, test_lbls)]
    
    #cache data
    torch.save(train_ds, os.path.join(args.data_cache_dir, train_file))
    torch.save(test_ds, os.path.join(args.data_cache_dir, test_file))

#Dataloader
train_dataloader = DataLoader(train_ds, shuffle=True, batch_size=args.batch_size, num_workers=0) 
val_dataloader =  DataLoader(test_ds, shuffle=True, batch_size=args.batch_size, num_workers=0)

/tmp/ipykernel_1062895/4174651498.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_ds = [{**example, 'token_labels': torch.tensor(tok_lbl, dtype=torch.long)} for example, tok_lbl in zip(train_ds, train_lbls)]
/tmp/ipykernel_1062895/4174651498.py:39: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_ds = [{**example, 'token_labels': torch.tensor(tok_lbl, dtype=torch.long)}for example, tok_lbl in zip(test_ds, test_lbls)]


## classification

In [6]:
#load model and tokenizer
print('loading model embedding model...')
tokenizer, embedding_model = load_tokenizer_embedding_model(args)
print('loading extraction model...')
ext_model = ClassificationModel(args, embedding_model)

loading model embedding model...


loading scibert model...
loading extraction model...


In [7]:
# log model setup
print(f'Logging run_setup')
def is_json_serializable(value):
    try:
        json.dumps(value)
        return True
    except (TypeError, OverflowError):
        return False

args_dict = vars(args)
filtered_args = {k: v for k, v in args_dict.items() if is_json_serializable(v)}
with open(os.path.join(args.output_dir, f'run_setup.json'), 'w') as f_out:
    json.dump(filtered_args, f_out, indent=4)

# log imput sample
print(f'Logging input sample')
input_sample = [tokenizer.convert_ids_to_tokens(ids=train_ds[i]['input_ids']) for i in range(3)]

with open(os.path.join(args.output_dir, f'input_sample.json'), 'w') as f_out:
    json.dump(input_sample, f_out, indent=4)
    
# print sample text
first_example = train_ds[0]
sample_text = tokenizer.convert_ids_to_tokens(ids=first_example['input_ids'])
print(f'First example input text: {sample_text}')
#print number of predicting targets
num_pred_targets = len([token for token in first_example['input_ids'] if token not in [tokenizer.cls_token_id, tokenizer.sep_token_id, tokenizer.pad_token_id]])
print(f'Num pred targets (cls: {tokenizer.cls_token}, sep: {tokenizer.sep_token}, pad: {tokenizer.pad_token}): {num_pred_targets}')
#print special token in example
special_token_ids = [token for token in first_example['input_ids'] if token in tokenizer.additional_special_tokens_ids]
print(f'Special tokens in input: {tokenizer.convert_ids_to_tokens(ids=special_token_ids)}')
#print labels in example
labels = first_example['token_labels'].tolist()
print(f'Labels {labels}')
labels = first_example['intent_labels'].tolist()
print(f'Labels {labels}')

Logging run_setup
Logging input sample
First example input text: ['[CLS]', 'before', 'we', 'turn', 'to', 'swedish', 'and', 'turkish', ',', 'focusing', 'on', 'recall', 'and', 'precision', 'of', 'root', 'nodes', ',', 'as', 'a', 'reflection', 'of', 'global', 'syntactic', 'structure', ',', 'and', 'on', 'attachment', 'score', 'as', 'a', 'function', 'of', 'arc', 'length', '.', 'if', 'we', 'start', 'by', 'considering', 'languages', 'with', 'a', 'labeled', 'attachment', 'score', 'of', '85', '%', 'or', 'higher', ',', 'they', 'are', 'characterized', 'by', 'high', 'precision', 'and', 'recall', 'for', 'root', 'nodes', ',', 'typically', '95', '/', '90', ',', 'and', 'by', 'a', 'grace', '##ful', 'degradation', 'of', 'attachment', 'score', 'as', 'arcs', 'grow', 'longer', ',', 'typically', '95', '##90', '##85', ',', 'for', 'arcs', 'of', 'length', '1', ',', '2', 'and', '36', '.', 'typical', 'examples', 'are', 'bul', '##gar', '##ian', '<', 'target', '_', 'citation', '/', '>', ',', 'chinese', '<', 'citati

In [8]:
from finecite.model import trainer
from importlib import reload
reload(trainer)

# start training
trainer = trainer.CustomTrainer(
    args=args,
    model=ext_model,
    tokenizer=tokenizer,
    train_dataloader=train_dataloader,
    test_dataloader=val_dataloader,
)
trainer.train()


configuring optimizer...

starting training...


{'epoch': 1, 'loss': 2.181, 'macro_f1': 0.2, 'micro_f1': 0.29, 'precision': [0.538, 0.154, 0.436, 0.0, 0.5, 0.061], 'recall': [0.171, 0.4, 0.654, 0.0, 0.048, 0.667], 'f1': [0.259, 0.222, 0.523, 0.0, 0.087, 0.111]}
{'epoch': 2, 'loss': 2.172, 'macro_f1': 0.31, 'micro_f1': 0.44, 'precision': [0.511, 0.333, 0.538, 0.0, 0.343, 0.25], 'recall': [0.561, 0.2, 0.269, 0.0, 0.571, 0.333], 'f1': [0.535, 0.25, 0.359, 0.0, 0.429, 0.286]}
{'epoch': 3, 'loss': 2.042, 'macro_f1': 0.287, 'micro_f1': 0.41, 'precision': [0.558, 0.0, 0.833, 0.0, 0.333, 0.333], 'recall': [0.585, 0.0, 0.192, 0.0, 0.476, 0.667], 'f1': [0.571, 0.0, 0.312, 0.0, 0.392, 0.444]}
{'current_epoch': 4, 'current_step': 100, 'avg_loss': 0.70298, 'max_loss': 1.3777, 'min_loss': 0.34143}
{'epoch': 4, 'loss': 2.204, 'macro_f1': 0.335, 'micro_f1': 0.53, 'precision': [0.667, 0.0, 0.8, 0.0, 0.371, 0.2], 'recall': [0.634, 0.0, 0.462, 0.0, 0.619, 0.667], 'f1': [0.65, 0.0, 0.585, 0.0, 0.464, 0.308]}
{'epoch': 5, 'loss': 2.432, 'macro_f1': 0.35